# 3. KNN

For the KNN algorithm, we will take a deeper look at how to generate synthetic data, something we are already using.
After, we will use the generated dataset to implement the algorithm and evaluate the results. Step by step, we will:
1. Create synthetic datasets consisting of two classes of objects;
2. Develop the k-NN algorithm;
3. Evaluate the algorithm's ability to separate the two classes of objects.

## Step 1. Create synthetic dataset with two different classes

Firstly, we need to generate training and validation/testing datasets for binary classification.
For visualisation purposes, we will assume that our objects have *2 numeric features*.
We would like to generate 2 "cloulds" of points on the plane corresponding to the positive and negative objects respectively. To do this, one can generate random points from a [multivariate normal distribution](https://en.wikipedia.org/wiki/Multivariate_normal_distribution) (function ``np.random.multivariate_normal``). For example, ``np.random.multivariate_normal([a,b], [[1,0],[0,1]], N)`` will generate a set on N points scattered around the *mean* point $(a,b)$.

1. Create two sets of N=20 points. The first set scattered around the point ``(1,1)`` and the second scattered around the point ``(3,3)``. These sets of points will correspond to the positive and the negative class respectively.

In [ ]:
import numpy as np

N = 20

In [ ]:
positive = np.random.multivariate_normal([1,1], [[1,0],[0,1]], N)
negative = np.random.multivariate_normal([3,3], [[1,0],[0,1]], N)

print(positive.shape, negative.shape)

2. Plot the generated sets of points. Use different colours or markers for different classes.

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(positive[:,0], positive[:,1], c='r')  # red
plt.scatter(negative[:,0], negative[:,1], c='g')  # green

plt.show()


3. Split each of the sets into equal train and validation portions. As a result you should have four sets:
- positive object/sample in the train dataset;
- positive object/sample in the validation dataset;
- negataive object/sample in the train dataset;
- negataive object/sample in the validation dataset;

To confirm that the sets have equal numbers of objects, print the number of elements in each set.

In [ ]:
halfN = int(N/2)
train_positive = positive[:halfN,:]
validation_positive = positive[halfN:,:]

train_negative = negative[:halfN,:]
validation_negative = negative[:halfN,:]

In [ ]:
print(len(train_positive), len(validation_positive), len(train_negative), len(validation_negative))

4. Add an extra freature (representing the class label: +1 for the positive class, -1 for the negative class) to the train and validation instances. As a result you will have two datasets, each consisting of tuples (label, instance).

In [ ]:
train_data = [(1, x) for x in train_positive]
train_data.extend([(-1, x) for x in train_negative])

validation_data = [(1, x) for x in validation_positive]
validation_data.extend([(-1, x) for x in validation_negative])

# To see that we have properly made tuples of (label, instance) lets print train and validation data.
print("Train data:\n", train_data)
print("\nValidation data:\n", validation_data)

## Step 2. Develop the k-NN algorithm

We use the training dataset from the previous step to implement k-NN prediction algorithm.
We choose cosine similarity as a "measure of distance".
The larger the similarity between two objects, the closer the objects are to each other.

1. Create the cosine similarity function that will be used in the k-NN prediction function to find the neighbours. The function should take two vectors as input and output the cosine similarity between the vectors: $\operatorname{cosSimilarity}(p,q)=\cos (\theta)=\frac{p \cdot q }{\|p\|\|q\|}$

In [ ]:
def cosSimilarity(p, q):
    score = np.dot(p,q) / (np.linalg.norm(p) * np.linalg.norm(q))
    return score

2. Implement a function that predicts the class of a validation instance using the k-NN algorithm. The function should take a validation instance and the parameter $k$ as input, and output predicted class of the validation instance (+1 or -1).

In [ ]:
def predict(x, k):  # x is the input, k is the number of neighbours
    # The first thing we do is computing the similarity scores between x and each instance z in our train dataset.
    # We must also store the labels so that we can later find the majority label.
    L = [(y, cosSimilarity(x, z)) for (y,z) in train_data]

    # Next, we need to find the neighbours.
    # For that we sort this list of tuples by the value of the second item in tuples, which is similarity.
    #
    # We use lambda expressions, which are convenient for writing in-place functions.
    # Here, we take an element from our list and return its similarity component,
    # which is used by the sort function to compare two elements.
    # "reverse=True" ensures that sorting is done in the descending order of similarity.
    L.sort(key=lambda a: a[1], reverse=True)

    # If you would like to confirm that it is indeed the descending order you can print the list after sorting (uncomment the line below).
    #print(L[:k])

    # Next, we must find the majority label.
    # Since we are doing binary classification and our labels are -1 and +1,
    # when we add the labels for the nearest neigbours if we get a positive value then there must be more +1s than -1s, and vice versa.
    # You might have to do more complicated stuff for finding the majority label if there were more than 2 classes.
    # But it is easy for the binary case as shown here.
    score = sum([e[0] for e in L[:k]])
    if score > 0:
        return 1
    else:
        return -1

## Step 3. Evaluate the algorithm

1. Implement ``kNNaccuracy`` function that takes the parameter ``k`` and the validation dataset as input and output the accuracy of the k-NN algorithm on the validation dataset. Use the function to compute the accuracy of prediciton of the k-NN classifier on the validation dataset, when ``k = 5``.

In [ ]:
def kNNaccuracy(k, validation_data):
    correctPredictions = 0
    for (y,x) in validation_data:  # for each validation sample
        if y == predict(x, k):  # get the prediction and compare with the correct label
            correctPredictions += 1
    accuracy = float(correctPredictions) / float(len(validation_data))
    return accuracy

print("Accuracy of 5-NN: ", kNNaccuracy(5, validation_data))

2. Now, let's compute accuracies of k-NN for all odd ``k`` from 1 to 99. Plotting k-NN accuracy versus ``k``, what is the best value of ``k`` for the validation dataset?

In [ ]:
kRange = range(1, 100, 2)
# Compute accuracies
accuracyValues = [kNNaccuracy(i, validation_data) for i in kRange]

# Plot the results
plt.plot(kRange, accuracyValues, 'b')
plt.xlabel("Parameter k")
plt.ylabel("Accuracy of k-NN")
plt.title("k-NN accuracy versus k")
plt.show()

# Find the best k for the current validation dataset
print("Best k: ", kRange[np.argmax(accuracyValues)])

# kNN using SKLearn

Although we implemented kNN from scratch, ``SKLearn`` has its own implementation of the method, which can be checked [here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).

Below we just separate the label from the data.

In [ ]:
# Extract labels and instances into separate vectors
train_labels = [item[0] for item in train_data]
train_instances = [item[1] for item in train_data]

print("Train Labels:", train_labels)
print("Train Instances:", train_instances)

val_labels = [item[0] for item in validation_data]
val_instances = [item[1] for item in validation_data]

print("Validation Labels:", val_labels)
print("Validation Instances:", val_instances)

Then, we can send the data and label to the kNN classifier.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=5)  # here we are using the euclidean distance, not the cosine

neigh.fit(train_instances, train_labels)

Finally, we can predict and calculate the accuracy

In [ ]:
y_pred = neigh.predict(val_instances)  # prediction

correctPredictions = 0
for real_label, pred_label in zip(val_labels, y_pred):
    if real_label == pred_label:
        correctPredictions += 1
accuracy = float(correctPredictions) / float(len(val_labels))
print(accuracy)